In [1]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup
import json

In [2]:
base_url ='https://understat.com/match/'
match = str(input("Enter Match ID"))
url=base_url+match
print(url)

Enter Match ID17161
https://understat.com/match/17161


In [3]:
res = rq.get(url)
soup = BeautifulSoup(res.content, "lxml") #parsing content of the page
scripts = soup.find_all('script')
print(scripts)

[<script>
			var THEME = localStorage.getItem("theme") || 'DARK';
			document.body.className = "theme-" + THEME.toLowerCase();
		</script>, <script>
	var shotsData 	= JSON.parse('\x7B\x22h\x22\x3A\x5B\x7B\x22id\x22\x3A\x22429019\x22,\x22minute\x22\x3A\x2210\x22,\x22result\x22\x3A\x22SavedShot\x22,\x22X\x22\x3A\x220.749000015258789\x22,\x22Y\x22\x3A\x220.23600000381469727\x22,\x22xG\x22\x3A\x220.0391988679766655\x22,\x22player\x22\x3A\x22Nabil\x20Fekir\x22,\x22h_a\x22\x3A\x22h\x22,\x22player_id\x22\x3A\x223667\x22,\x22situation\x22\x3A\x22DirectFreekick\x22,\x22season\x22\x3A\x222021\x22,\x22shotType\x22\x3A\x22LeftFoot\x22,\x22match_id\x22\x3A\x2217161\x22,\x22h_team\x22\x3A\x22Real\x20Betis\x22,\x22a_team\x22\x3A\x22Real\x20Madrid\x22,\x22h_goals\x22\x3A\x220\x22,\x22a_goals\x22\x3A\x221\x22,\x22date\x22\x3A\x222021\x2D08\x2D28\x2020\x3A00\x3A00\x22,\x22player_assisted\x22\x3Anull,\x22lastAction\x22\x3A\x22Standard\x22\x7D,\x7B\x22id\x22\x3A\x22429020\x22,\x22minute\x22\x3A\x2220\x22,

In [4]:
strings = scripts[1].string
#strip unnecessary symbols 
index_start=strings.index("('")+2
index_end=strings.index("')")
dataset= strings[index_start:index_end]
dataset = dataset.encode('utf8').decode('unicode_escape')

In [5]:
data = json.loads(dataset)

In [6]:
data

{'h': [{'id': '429019',
   'minute': '10',
   'result': 'SavedShot',
   'X': '0.749000015258789',
   'Y': '0.23600000381469727',
   'xG': '0.0391988679766655',
   'player': 'Nabil Fekir',
   'h_a': 'h',
   'player_id': '3667',
   'situation': 'DirectFreekick',
   'season': '2021',
   'shotType': 'LeftFoot',
   'match_id': '17161',
   'h_team': 'Real Betis',
   'a_team': 'Real Madrid',
   'h_goals': '0',
   'a_goals': '1',
   'date': '2021-08-28 20:00:00',
   'player_assisted': None,
   'lastAction': 'Standard'},
  {'id': '429020',
   'minute': '20',
   'result': 'SavedShot',
   'X': '0.76',
   'Y': '0.40900001525878904',
   'xG': '0.022661922499537468',
   'player': 'Guido Rodríguez',
   'h_a': 'h',
   'player_id': '8267',
   'situation': 'OpenPlay',
   'season': '2021',
   'shotType': 'RightFoot',
   'match_id': '17161',
   'h_team': 'Real Betis',
   'a_team': 'Real Madrid',
   'h_goals': '0',
   'a_goals': '1',
   'date': '2021-08-28 20:00:00',
   'player_assisted': 'Sergio Canales',

In [7]:
x=[]
y=[]
xG=[]
s=[]
team=[]
result=[]
data_away=data['a']
data_home=data['h']
for index in range(len(data_home)):
    for key in data_home[index]:
        if key == 'X':
            x.append(data_home[index][key])
        if key == 'Y':
            y.append(data_home[index][key])
        if key == 'h_team':
            team.append(data_home[index][key])
        if key == 'xG':
            xG.append(data_home[index][key])
        if key == 'result':
            result.append(data_home[index][key])
        if key == 'situation':
            s.append(data_home[index][key])

for index in range(len(data_away)):
    for key in data_away[index]:
        if key == 'X':
            x.append(data_away[index][key])
        if key == 'Y':
            y.append(data_away[index][key])
        if key == 'a_team':
            team.append(data_away[index][key])
        if key == 'xG':
            xG.append(data_away[index][key])
        if key == 'result':
            result.append(data_away[index][key])
        if key == 'situation':
            s.append(data_away[index][key])

In [8]:
columnNames=['Team','Situation','Result','X','Y','xG']
df = pd.DataFrame([team,s,result,x,y,xG],index=columnNames)
df = df.T

In [9]:
df

,Team,Situation,Result,X,Y,xG
0,Real Betis,DirectFreekick,SavedShot,0.749000015258789,0.23600000381469727,0.0391988679766655
1,Real Betis,OpenPlay,SavedShot,0.76,0.40900001525878904,0.022661922499537468
2,Real Betis,OpenPlay,MissedShots,0.7559999847412109,0.5120000076293946,0.02578043006360531
3,Real Betis,OpenPlay,MissedShots,0.805,0.28399999618530275,0.024953028187155724
4,Real Betis,OpenPlay,SavedShot,0.7719999694824219,0.3029999923706055,0.019759133458137512
5,Real Betis,OpenPlay,BlockedShot,0.799000015258789,0.6619999694824219,0.02599579282104969
6,Real Betis,OpenPlay,MissedShots,0.909000015258789,0.48200000762939454,0.09513796865940094
7,Real Betis,FromCorner,SavedShot,0.8480000305175781,0.26299999237060545,0.029286973178386688
8,Real Betis,OpenPlay,SavedShot,0.775,0.644000015258789,0.02262577973306179
9,Real Betis,OpenPlay,MissedShots,0.7090000152587891,0.55,0.012270641513168812


In [10]:
df.to_csv(match+'.csv')